In [10]:
import wandb
from datasets import load_dataset, DatasetDict


dataset: DatasetDict = load_dataset("ai-forever/spellcheck_benchmark", "RUSpellRU")


punct_dataset: DatasetDict = load_dataset(
    "ai-forever/spellcheck_punctuation_benchmark", "RUSpellRU", trust_remote_code=True
)

Using the latest cached version of the dataset since ai-forever/spellcheck_benchmark couldn't be found on the Hugging Face Hub
Using the latest cached version of the dataset since ai-forever/spellcheck_benchmark couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'RUSpellRU' at C:\Users\Андрей Т\.cache\huggingface\datasets\ai-forever___spellcheck_benchmark\RUSpellRU\0.0.1\3395aa540689e4393c3e18d063e73a5b99d7f047 (last modified on Mon Jun 17 00:55:50 2024).
Found the latest cached dataset configuration 'RUSpellRU' at C:\Users\Андрей Т\.cache\huggingface\datasets\ai-forever___spellcheck_benchmark\RUSpellRU\0.0.1\3395aa540689e4393c3e18d063e73a5b99d7f047 (last modified on Mon Jun 17 00:55:50 2024).


In [1]:
import language_tool_python
from src.base import BaseModel


from typing import List, Tuple, Dict


class PyLanguageToolModel(BaseModel):

    def __init__(self):
        super().__init__()
        self.language_tool = language_tool_python.LanguageTool("ru")

    def predict_verbose(self, text: str) -> List[Tuple[str, List[str]]]:
        matches = self.language_tool.check(text)
        corrections = []
        for match in matches:
            error = text[match.offset : match.offset + match.errorLength + 1]
            error = error.replace(" ", "")
            corrections.append(
                {
                    "index": match.offset,
                    "error": error,
                    "suggestions": match.replacements,
                    "message": match.message,
                }
            )
            try:
                repl = match.replacements[0]
            except IndexError:
                repl = error
            text = text.replace(error, repl)
        return corrections, text

    def predict(self, text: str):
        _, result = self.predict_verbose(text)
        return result

    def evaluate(self, text: str, answer: str) -> Dict:
        raise NotImplementedError
        return super().evaluate(text, answer)

In [12]:
pmt = PyLanguageToolModel()
pmt.predict("Превет")

'Привет'

Скоринг по орфографии

In [13]:
from src.model_scorers import WandbSageModelScorer

sms = WandbSageModelScorer(dataset=dataset["test"], project="language_tool_python")

In [14]:
scoring_final_result, explanation = sms.score_explain(
    pmt, metrics=["errant", "ruspelleval"]
)

Calculating words metric: 100%|██████████| 2008/2008 [00:14<00:00, 137.72it/s]


In [15]:
scoring_final_result

{'CASE_Precision': 0.0,
 'CASE_Recall': 100.0,
 'CASE_F1': 0.0,
 'SPELL_Precision': 36.53,
 'SPELL_Recall': 32.75,
 'SPELL_F1': 34.53,
 'YO_Precision': 0.0,
 'YO_Recall': 100.0,
 'YO_F1': 0.0,
 'PUNCT_Precision': 0.0,
 'PUNCT_Recall': 0.0,
 'PUNCT_F1': 0.0,
 'Precision': 45.2,
 'Recall': 35.49,
 'F1': 39.76}

In [16]:
explanation

,Source,Truth,Model_result,Model_is_correct
0,﻿есть у вас оформленый и подписаный мною заказ,﻿есть у вас оформленный и подписанный мною заказ,﻿Есть у вас оформленный и подписанный мною заказ,True
1,вот в инете откапал такую интеерсную статейку ...,вот в инете откопал такую интересную статейку ...,Вот в иннете откапал такую интересную статейку...,False
2,я на всю жизнь запомню свое первое купание в з...,я на всю жизнь запомню свое первое купание в з...,Я на всю жизнь запомню свое первое купание в з...,True
3,думаем что не ошибемся если скажем что выставк...,думаем что не ошибемся если скажем что выставк...,Думаем что не ошибемся если скажем что выставк...,True
4,судьба человека может складываться очень разно...,судьба человека может складываться очень разно...,Судьба человека может складываться очень разно...,True
...,...,...,...,...
2003,спасибо вам огромное за нормальную новость о е...,спасибо вам огромное за нормальную новость о е...,Спасибо вам огромное за нормальную новость о и...,False
2004,более захватывающее и наглядное обучение возмо...,более захватывающее и наглядное обучение возмо...,Более захватывающее и наглядное обучение возмо...,True
2005,и вобщем-то все понятно на фоне слухов застави...,и в общем-то все понятно на фоне слухов застав...,И общем-то все понятно на фоне слухов заставИл...,False
2006,но всему есть придел,но всему есть предел,Но всему есть придел,False


In [17]:
acc = f"Accuracy: {len(explanation[explanation.Model_is_correct]) / len(explanation)}"
acc

'Accuracy: 0.475597609561753'

Скоринг пунктуации

In [18]:
sms.dataset = punct_dataset["test"]
sms.run_suffix = "_punct"


scoring_final_result, explanation = sms.score_explain(pmt, metrics=["errant"])

Calculating errant metric: 100%|██████████| 2008/2008 [05:51<00:00,  5.72it/s]


In [19]:
scoring_final_result

{'SPELL_Precision': 37.56,
 'SPELL_Recall': 26.17,
 'SPELL_F1': 30.84,
 'CASE_Precision': 66.27,
 'CASE_Recall': 78.56,
 'CASE_F1': 71.9,
 'PUNCT_Precision': 41.94,
 'PUNCT_Recall': 0.22,
 'PUNCT_F1': 0.45,
 'YO_Precision': 3.45,
 'YO_Recall': 0.28,
 'YO_F1': 0.51}

In [20]:
explanation

,Source,Truth,Model_result,Model_is_correct
0,а так хочеться что-то мочь менять в этом мире ...,А так хочется что-то мочь менять в этом мире: ...,А тАк хочется что-то мочь менять в этом мире н...,False
1,давольно милый и летом и зимой обогреваемый те...,"Довольно милый, и летом, и зимой обогреваемый ...",Довольно милый и летом и зимой обогреваемый те...,False
2,бывают такие моменты когда хочеться зделать чт...,"Бывают такие моменты, когда хочется сделать чт...",Бывают такие моменты когда хочется зделать что...,False
3,﻿есть у вас оформленый и подписаный мною заказ,﻿Есть у вас оформленный и подписанный мною заказ?,﻿Есть у вас оформленный и подписанный мною заказ,False
4,вот в инете откапал такую интеерсную статейку ...,"Вот в инете откопал такую интересную статейку,...",Вот в иннете откапал такую интересную статейку...,False
...,...,...,...,...
2003,спасибо вам огромное за нормальную новость о е...,Спасибо вам огромное за нормальную новость о Ё...,Спасибо вам огромное за нормальную новость о и...,False
2004,более захватывающее и наглядное обучение возмо...,Более захватывающее и наглядное обучение возмо...,Более захватывающее и наглядное обучение возмо...,False
2005,и вобщем-то все понятно на фоне слухов застави...,"И, в общем-то, всё понятно на фоне слухов: зас...",И общем-то все понятно на фоне слухов заставИл...,False
2006,но всему есть придел,Но всему есть предел.,Но всему есть придел,False


In [21]:
f"Punct Accuracy: {len(explanation[explanation.Model_is_correct]) / len(explanation)}"

'Punct Accuracy: 0.0'

In [22]:
wandb.run.finish()

# Выводы

https://wandb.ai/obuchii/NLP%20Scoring/runs/lkrlnnk2

|  Задача | Errant Precision | Errant Recall | Errant F1 | Precision | Recall | F1
|----------|----------|----------|----------|----------|----------|----------|
| Spelling   | 36.53   | 32.75   | 34.53    | 45.2   | 35.49   | 39.76    |

-------


|  Задача | Punct Precision | Punct Recall | Punct F1 | Case Precision | Case Recall | Case F1
|----------|----------|----------|----------|----------|----------|----------|
| Punctuation   | 41.94   | 0.22   | 0.45    | 66.27  | 78.56   | 71.9    |


Модель плохенько справляется со спеллингом, обычно на сложных примерах.

Модель не умеет исправлять пунктуационные ошибки.